# Making a new material file

**Optional**: build protocol buffer package

In [1]:
!protoc --python_out=. -I=../proto ../proto/material.proto 

Import library. Note: if you get an error that says "no module named google," make sure you have protobuf python library installed (try `pip install protobuf`)

In [1]:
import material_pb2 as mat

In [15]:
c5g7_moderator = mat.Material()
c5g7_moderator.full_name = "C5G7 Moderator"
c5g7_moderator.id = "c5g7_mod"
c5g7_moderator.abbreviation = "c567_mod"
c5g7_moderator.is_fissionable = False
c5g7_moderator.number_of_groups = 7

e_max = 1e6
energy_groups = [e_max, 1e3, 1e2, 0.625, 0.625/2, 0.625/4, 0]
eg = mat.Material.VectorProperty()
eg.id = mat.Material.ENERGY_GROUPS
eg.value.extend(energy_groups)

sigma_t = mat.Material.VectorProperty()
sigma_t.id = mat.Material.SIGMA_T
sigma_t.value.extend([.240, .975, 3.1, 3.1, 0.975, 0.240])

nsf = mat.Material.VectorProperty()
nsf.id = mat.Material.NU_SIG_F
nsf.value.extend([3 * 0.006, 2.5 * 0.060, 2.0 * 0.9, 2 * 0.9, 2.5 * 0.060, 3 * 0.006])

chi = mat.Material.VectorProperty()
chi.id = mat.Material.CHI
chi.value.extend([.48, 0.02, 0, 0, 0.02, 0.48])

sigma_s = mat.Material.MatrixProperty()
sigma_s.id = mat.Material.SIGMA_S
sigma_s.value.extend([0.0240, 0,     0,     0,     0,     0,
                      0.171,  0.6,   0,     0,     0,     0,
                      0.033,  0.275, 2.0,   0,     0,     0,
                      0,      0,     0,     2.0,   0.275, 0.033,
                      0,      0,     0,     0,     0.6,   0.171,
                      0,      0,     0,     0,     0,     0.0240])

c5g7_moderator.vector_property.extend([eg, sigma_t, nsf, chi])
c5g7_moderator.matrix_property.extend([sigma_s])

filename = '/home/josh/repos/bart/benchmarks/sood_1999/uc5g7_moderator_6_0_in/research_reactor_6_group.material'
f = open(filename, 'wb')
f.write(c5g7_moderator.SerializeToString())
f.close()

In [3]:
rr = mat.Material()
rr.full_name = "Uranium research reactor, 6 group"
rr.id = "URR_6"
rr.abbreviation = "URR_6"
rr.is_fissionable = True
rr.number_of_groups = 6

e_max = 1e6
energy_groups = [e_max, 1e3, 1e2, 0.625, 0.625/2, 0.625/4, 0]
eg = mat.Material.VectorProperty()
eg.id = mat.Material.ENERGY_GROUPS
eg.value.extend(energy_groups)

sigma_t = mat.Material.VectorProperty()
sigma_t.id = mat.Material.SIGMA_T
sigma_t.value.extend([.240, .975, 3.1, 3.1, 0.975, 0.240])

nsf = mat.Material.VectorProperty()
nsf.id = mat.Material.NU_SIG_F
nsf.value.extend([3 * 0.006, 2.5 * 0.060, 2.0 * 0.9, 2 * 0.9, 2.5 * 0.060, 3 * 0.006])

chi = mat.Material.VectorProperty()
chi.id = mat.Material.CHI
chi.value.extend([.48, 0.02, 0, 0, 0.02, 0.48])

sigma_s = mat.Material.MatrixProperty()
sigma_s.id = mat.Material.SIGMA_S
sigma_s.value.extend([0.0240, 0,     0,     0,     0,     0,
                      0.171,  0.6,   0,     0,     0,     0,
                      0.033,  0.275, 2.0,   0,     0,     0,
                      0,      0,     0,     2.0,   0.275, 0.033,
                      0,      0,     0,     0,     0.6,   0.171,
                      0,      0,     0,     0,     0,     0.0240])

rr.vector_property.extend([eg, sigma_t, nsf, chi])
rr.matrix_property.extend([sigma_s])

filename = '/home/josh/repos/bart/benchmarks/sood_1999/urr_6_0_in/research_reactor_6_group.material'
f = open(filename, 'wb')
f.write(rr.SerializeToString())
f.close()

In [32]:
def process_material_data(material_data):
    return_material = mat.Material()
    return_material.full_name = material_data['name']
    return_material.id = material_data['id']
    return_material.abbreviation = material_data['abbrev']
    return_material.is_fissionable = material_data['is_fissionable']
    return_material.number_of_groups = material_data['n_groups']

    energy_groups = mat.Material.VectorProperty()
    energy_groups.id = mat.Material.ENERGY_GROUPS
    energy_groups.value.extend(material_data['energy_groups'])

    sigma_t = mat.Material.VectorProperty()
    sigma_t.id = mat.Material.SIGMA_T
    sigma_t.value.extend(material_data['sigma_t'])

    sigma_s = mat.Material.MatrixProperty()
    sigma_s.id = mat.Material.SIGMA_S
    sigma_s.value.extend(material_data['sigma_s'])

    if (material_data['is_fissionable']):
        nu_sig_f = mat.Material.VectorProperty()
        nu_sig_f.id = mat.Material.NU_SIG_F
        nu_sig_f.value.extend(material_data['nu_sig_f'])

        chi = mat.Material.VectorProperty()
        chi.id = mat.Material.CHI
        chi.value.extend(material_data['chi'])

        return_material.vector_property.extend([energy_groups, sigma_t, nu_sig_f, chi])
        return_material.matrix_property.extend([sigma_s])
    else:
        return_material.vector_property.extend([energy_groups, sigma_t])
        return_material.matrix_property.extend([sigma_s])
    return return_material

In [37]:
filename = '/home/josh/repos/bart/benchmarks/sood_1999/problem_64/table_46_research_reactor.material'

rr_data = {"name" : "Research reactor, Table 46/47",
           "id" : "RR_46_47",
           "abbrev" : "RR_46_47",
           "is_fissionable" : True,
           "n_groups" : 2,
           "energy_groups" : [10000000.0, 0.625, 0.0],
           "sigma_t" : [0.650917, 2.138],
           "sigma_s" : [0.0, 0.0, 0.0342008, 2.06880],
           "nu_sig_f" : [1.004*.61475, 2.5*0.045704],
           "chi" : [1.0, 0]
}
rr_material = process_material_data(rr_data)
f = open(filename, 'wb')
f.write(rr_material.SerializeToString())
f.close()

In [38]:
filename = '/home/josh/repos/bart/benchmarks/sood_1999/problem_64/table_46_h20_b.material'

h20_data = {"name" : "H20b, Table 46/47",
           "id" : "H20b_46_47",
           "abbrev" : "H20B_46_47",
           "is_fissionable" : False,
           "n_groups" : 2,
           "energy_groups" : [10000000.0, 0.625, 0.0],
           "sigma_t" : [0.1106832906, 0.36355],
           "sigma_s" : [0.1096742149, 0.0, 0.001000595707, 0.36339],
    }
h20_material = process_material_data(rr_data)
f = open(filename, 'wb')
f.write(h20_material.SerializeToString())
f.close()

In [18]:
material_b = mat.Material()
material_b.full_name = "Multi-slab Material B"
material_b.id = "mat_b"
material_b.abbreviation = "mat_b"
material_b.is_fissionable = True
material_b.number_of_groups = 1

energy_groups = mat.Material.VectorProperty()
energy_groups.id = mat.Material.ENERGY_GROUPS
energy_groups.value.extend([10000000.0, 0.0])

sigma_t = mat.Material.VectorProperty()
sigma_t.id = mat.Material.SIGMA_T
sigma_t.value.extend([1])

sigma_s = mat.Material.MatrixProperty()
sigma_s.id = mat.Material.SIGMA_S
sigma_s.value.extend([0.4])

nu_sig_f = mat.Material.VectorProperty()
nu_sig_f.id = mat.Material.NU_SIG_F
nu_sig_f.value.extend([.9])

chi = mat.Material.VectorProperty()
chi.id = mat.Material.CHI
chi.value.extend([1])

material_b.vector_property.extend([energy_groups, sigma_t, nu_sig_f, chi])
material_b.matrix_property.extend([sigma_s])
print(material_b)

f = open('/home/josh/repos/bart/benchmarks/picca_2016/multi_slab/material_b', 'wb')
f.write(material_b.SerializeToString())
f.close()

full_name: "Multi-slab Material B"
abbreviation: "mat_b"
id: "mat_b"
number_of_groups: 1
is_fissionable: true
vector_property {
  id: ENERGY_GROUPS
  value: 10000000.0
  value: 0.0
}
vector_property {
  id: SIGMA_T
  value: 1.0
}
vector_property {
  id: NU_SIG_F
  value: 0.9
}
vector_property {
  id: CHI
  value: 1.0
}
matrix_property {
  id: SIGMA_S
  value: 0.4
}



In [45]:
material_a = mat.Material()
material_a.full_name = "Multi-slab Material A"
material_a.id = "mat_a"
material_a.abbreviation = "mat_a"
material_a.is_fissionable = False
material_a.number_of_groups = 1

energy_groups = mat.Material.VectorProperty()
energy_groups.id = mat.Material.ENERGY_GROUPS
energy_groups.value.extend([10000000.0, 0.0])

sigma_t = mat.Material.VectorProperty()
sigma_t.id = mat.Material.SIGMA_T
sigma_t.value.extend([1])

sigma_s = mat.Material.MatrixProperty()
sigma_s.id = mat.Material.SIGMA_S
sigma_s.value.extend([0.5])

material_a.vector_property.extend([energy_groups, sigma_t])
material_a.matrix_property.extend([sigma_s])
print(material_a)

f = open('/home/josh/repos/bart/benchmarks/picca_2016/multi_slab/material_a', 'wb')
f.write(material_a.SerializeToString())
f.close()

full_name: "Multi-slab Material A"
abbreviation: "mat_a"
id: "mat_a"
number_of_groups: 1
vector_property {
  id: ENERGY_GROUPS
  value: 10000000.0
  value: 0.0
}
vector_property {
  id: SIGMA_T
  value: 1.0
}
matrix_property {
  id: SIGMA_S
  value: 0.5
}



In [40]:



rx.matrix_property[0].value[0] = 0.099
filename = '/home/josh/repos/bart/benchmarks/picca_2016/multi_slab/material_a'
f = open(filename, 'wb')
f.write(rx.SerializeToString())
f.close()

In [17]:
water = mat.Material()
water.full_name = "h20"
water.id = "h20"
water.abbreviation = "h20"
water.is_fissionable = False
water.number_of_groups = 2

energy_groups = mat.Material.VectorProperty()
energy_groups.id = mat.Material.ENERGY_GROUPS
energy_groups.value.extend([10000000.0, 0.625, 0.0])

sigma_t = mat.Material.VectorProperty()
sigma_t.id = mat.Material.SIGMA_T
sigma_t.value.extend([0.00074, 0.018564])

sigma_s = mat.Material.MatrixProperty()
sigma_s.id = mat.Material.SIGMA_S
sigma_s.value.extend([0.83975, 0.000336, 0.04749, 2.9676])

water.vector_property.extend([energy_groups, sigma_t])
water.matrix_property.extend([sigma_s])

f = open('/home/josh/repos/bart/benchmarks/test_problems/3d_half_box/water.material', 'wb')
f.write(water.SerializeToString())
f.close()

In [3]:
reflector = mat.Material()
reflector.full_name = "Reflector"
reflector.id = "reflector"
reflector.abbreviation = "reflector"
reflector.is_fissionable = False
reflector.number_of_groups = 2

energy_groups = mat.Material.VectorProperty()
energy_groups.id = mat.Material.ENERGY_GROUPS
energy_groups.value.extend([10000000.0, 0.625, 0.0])

sigma_t = mat.Material.VectorProperty()
sigma_t = mat.Material.SIGMA_T
sigma_t.value.extend([0.1, 0.1])

sigma_s = mat.Material.MatrixProperty()
sigma_s = mat.Material.SIGMA_S
sigma_s.value.extend([])

In [40]:
medium = mat.Material()

medium.full_name = "Medium (Benchmark 3.2, Ganapol 2009)"
medium.id = "medium"
medium.abbreviation = "medium"
medium.is_fissionable = False
medium.number_of_groups = 1

energy_groups = mat.Material.VectorProperty()
energy_groups.id = mat.Material.ENERGY_GROUPS
energy_groups.value.extend([20000000.0, 0.0])

#diff = mat.Material.VectorProperty()
#diff.id = mat.Material.DIFFUSION_COEFF
#diff.value.extend([1/(3*.15)])

sigma_t_val = 100
c = 0.7

sigma_t = mat.Material.VectorProperty()
sigma_t.id = mat.Material.SIGMA_T
sigma_t.value.extend([sigma_t_val])

diff = mat.Material.VectorProperty()
diff.id = mat.Material.DIFFUSION_COEFF
diff.value.extend([1/(3*sigma_t_val)])

q = mat.Material.VectorProperty()
q.id = mat.Material.Q
q.value.extend([0])

# nu_sig_f = mat.Material.VectorProperty()
# nu_sig_f.id = mat.Material.NU_SIG_F
# nu_sig_f.value.extend([0.09])

# chi = mat.Material.VectorProperty()
# chi.id = mat.Material.CHI
# chi.value.extend([1.0])

sigma_s = mat.Material.MatrixProperty()
sigma_s.id = mat.Material.SIGMA_S
sigma_s.value.extend([c * sigma_t_val])

#medium.vector_property.extend([energy_groups, sigma_t, nu_sig_f, chi, diff])
medium.vector_property.extend([energy_groups, sigma_t, diff, q])
medium.matrix_property.extend([sigma_s])

print(medium)

full_name: "Medium (Benchmark 3.2, Ganapol 2009)"
abbreviation: "medium"
id: "medium"
number_of_groups: 1
vector_property {
  id: ENERGY_GROUPS
  value: 20000000.0
  value: 0.0
}
vector_property {
  id: SIGMA_T
  value: 100.0
}
vector_property {
  id: DIFFUSION_COEFF
  value: 0.0033333333333333335
}
vector_property {
  id: Q
  value: 0.0
}
matrix_property {
  id: SIGMA_S
  value: 70.0
}



In [41]:
f = open('/home/josh/repos/bart/benchmarks/ganapol_2009/3.2/medium.material', 'wb')
f.write(medium.SerializeToString())
f.close()

In [32]:
source = mat.Material()

source.full_name = "source (Benchmark 3.2, Ganapol 2009)"
source.id = "source"
source.abbreviation = "source"
source.is_fissionable = False
source.number_of_groups = 1

energy_groups = mat.Material.VectorProperty()
energy_groups.id = mat.Material.ENERGY_GROUPS
energy_groups.value.extend([20000000.0, 0.0])

sigma_t = mat.Material.VectorProperty()
sigma_t.id = mat.Material.SIGMA_T
sigma_t.value.extend([1])

q = mat.Material.VectorProperty()
q.id = mat.Material.Q
q.value.extend([0.5])

diff = mat.Material.VectorProperty()
diff.id = mat.Material.DIFFUSION_COEFF
diff.value.extend([1/(3)])

# nu_sig_f = mat.Material.VectorProperty()
# nu_sig_f.id = mat.Material.NU_SIG_F
# nu_sig_f.value.extend([0.0])

# chi = mat.Material.VectorProperty()
# chi.id = mat.Material.CHI
# chi.value.extend([0.0])

sigma_s = mat.Material.MatrixProperty()
sigma_s.id = mat.Material.SIGMA_S
sigma_s.value.extend([1e-16])


# source.vector_property.extend([diff, energy_groups, sigma_t, nu_sig_f, chi])
source.vector_property.extend([energy_groups, sigma_t, q, diff])
source.matrix_property.extend([sigma_s])

print(source)

full_name: "source (Benchmark 3.2, Ganapol 2009)"
abbreviation: "source"
id: "source"
number_of_groups: 1
vector_property {
  id: ENERGY_GROUPS
  value: 20000000.0
  value: 0.0
}
vector_property {
  id: SIGMA_T
  value: 1.0
}
vector_property {
  id: Q
  value: 0.5
}
vector_property {
  id: DIFFUSION_COEFF
  value: 0.3333333333333333
}
matrix_property {
  id: SIGMA_S
  value: 1e-16
}



In [33]:
f = open('/home/josh/repos/bart/benchmarks/ganapol_2009/3.2/source.material', 'wb')
f.write(source.SerializeToString())
f.close()